<a href="https://colab.research.google.com/github/jsvon/NLP-projects-/blob/main/Multi_Agent_Systems_using_Deepseek5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

MULTI AGENT SYSTEMS

In [2]:
!pip install 'smolagents[litellm]' plotly geopandas shapely kaleido -q


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 113.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 102.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.6/104.6 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 34.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 93.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 64.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.2.3 which is incompatible.


In [3]:
import math
from typing import Optional, Tuple

from smolagents import tool, DuckDuckGoSearchTool


@tool
def calculate_cargo_travel_time(
    origin_coords: Tuple[float, float],
    destination_coords: Tuple[float, float],
    cruising_speed_kmh: Optional[float] = 750.0,  # Average speed for cargo planes
) -> float:
    """
    Calculate the travel time for a cargo plane between two points on Earth using great-circle distance.

    Args:
        origin_coords: Tuple of (latitude, longitude) for the starting point
        destination_coords: Tuple of (latitude, longitude) for the destination
        cruising_speed_kmh: Optional cruising speed in km/h (defaults to 750 km/h for typical cargo planes)

    Returns:
        float: The estimated travel time in hours

    Example:
        >>> # Chicago (41.8781° N, 87.6298° W) to Sydney (33.8688° S, 151.2093° E)
        >>> result = calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093))
    """

    def to_radians(degrees: float) -> float:
        return degrees * (math.pi / 180)

    # Extract coordinates
    lat1, lon1 = map(to_radians, origin_coords)
    lat2, lon2 = map(to_radians, destination_coords)

    # Earth's radius in kilometers
    EARTH_RADIUS_KM = 6371.0

    # Calculate great-circle distance using the haversine formula
    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = (
        math.sin(dlat / 2) ** 2
        + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    )
    c = 2 * math.asin(math.sqrt(a))
    distance = EARTH_RADIUS_KM * c

    # Add 10% to account for non-direct routes and air traffic controls
    actual_distance = distance * 1.1

    # Calculate flight time
    # Add 1 hour for takeoff and landing procedures
    flight_time = (actual_distance / cruising_speed_kmh) + 1.0

    # Format the results
    return round(flight_time, 2)


print(calculate_cargo_travel_time((41.8781, -87.6298), (-33.8688, 151.2093)))

22.82


In [4]:
#setting up the agent

import os
from PIL import Image
from smolagents.models import HfApiModel # Import HfApiModel from smolagents.models
from smolagents import CodeAgent, DuckDuckGoSearchTool, VisitWebpageTool

model = HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud')

In [5]:
task = """
     Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.
Also give me some supercar factories with the same cargo plane transfer time.
"""

In [6]:
agent = CodeAgent(
    model = model,
    tools = [
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time
        ],
        additional_authorized_imports = ["pandas"],
        max_steps = 20,
)

In [10]:
result = agent.run(task)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                                │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  batman_locations_query = "Batman filming locations"                                                              
  batman_locations_search = web_search(query=batman_locations_query)                                               
  print(batman_locations_search)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Where The Batman Filmed (All Gotham Real-Life Locations) - Screen 
Rant](https://screenrant.com/where-was-the-batman-movie-filmed/)
The London location known as Two Temple Place served as another The Batman filming location. Two Temple Place was 
the first building seen in the movie, in fact, as it was the home of Gotham's Mayor Don Mitchell. Matt Reeves began
The Batman by showing the location through Riddler's eyes. Two Temple Place then became a major location for the 
...

[Where was The Batman filmed? ALL the Filming Locations in Chicago & The 
UK](https://www.atlasofwonders.com/2022/04/where-was-the-batman-filmed.html)
The Batman is the newest addition to the Caped Crusader films franchise. Matt Reeves, who also directed the last 
two films in the Planet of the Apes movie series, has made a more noir, detective-driven version of the story. The 
Batman was primarily filmed in the United Kingdom.Most of the recognizable buildings of this new version of Gotham 
City are located in Liverpool, although Chicago was a ...

[The Batman | Film Locations](https://www.movie-locations.com/movies/b/The-Batman-2022.php)
The Batman location: the subterranean entrance to 'Wayne Tower': Holborn tram tunnel, Holborn It's accessed via a 
subterranean passage, which is the old Kingsway Tram Tunnel , disused since trams were discontinued in London in 
1952, smack in the middle of busy Southampton Row just south of Theobald's Road near Holborn tube station.

[The Batman Filming Locations: Where was Matt Reeves Directorial 
Filmed?](https://www.theartistree.fm/entertainment/449883/the-batman-filming-locations-where-was-matt-reeves-direct
orial-filmed/)
For The Batman, Matt Reeves mainly filmed in the UK and Chicago to capture the cold, gothic, and older feel of 
Gotham. While The Batman did try to film in real locations, iconic spots like the Batcave and Wayne Tower were 
built on studio sets. When filming on location, the cast and crew spent most of their time in the UK for the main 
Gotham scenes.

[Film locations for Tim Burton's Batman (1989), in London, Hertfordshire 
...](https://movie-locations.com/movies/b/Batman-1989.php)
Film locations for Tim Burton's Batman in London and Southeast England: Knebworth House, Knebworth, Hertfordshire 
('Wayne Manor' exterior); Hatfield House, Hatfield, Hertfordshire ('Wayne Manor' interior); Pinewood Studio, 
Buckinghamshire (studio); Ladies' Smoking Room, St Pancras Renaissance Hotel, Euston Road, NW1 (office of Gotham 
Globe); Little Barford Power Station, St Neots ...

[12 Batman Movie Locations You Can Visit! - Leisure Asia 
Global](https://www.travelandleisureasia.com/global/destinations/batman-movie-locations-you-can-visit/)
Image: Courtesy Rob Farrow / Two, Temple Place, London/ Creative Commons Attribution-Share Alike 2.0 Generic/ 
Wikimedia Commons The stunning Victorian mansion in London is another officially IMDb listed filming location for 
Matt Reeves' The Batman.The building is as gorgeous as it can be and retains the classic European charm till date. 
The venue has been featured in several other noted ...

[These Are the Cities Standing in for Gotham in 'The Batman' - 
Distractify](https://www.distractify.com/p/the-batman-filming-locations)
The latest film centered around the Caped Crusader, The Batman, is finally in theaters.One of the key elements of 
any good Batman story is the Gotham City setting. Over the years, various directors have chosen different cities to
stand in for the crime-riddled location, but where did Matt Reeves set his epic Batman tale?

[The Batman Filming Locations: Where The Batcave Is Located? - 
OtakuKart](https://otakukart.com/the-batman-filming-locations-where-the-batcave-is-located/)
Marshall Suloway Bridge, situated in Chicago, became a part of the Filming locations of The Batman when we saw 
Bruce riding his motorcycle on the Bridge. Even Marshall Suloway Bridge is one of the historical locations in 
Chicago because it 

[Step 1: Duration 6.30 seconds| Input tokens: 2,379 | Output tokens: 190]

...

[Step 3: Duration 15.90 seconds| Input tokens: 14,793 | Output tokens: 897]

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 4 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Supercar factories and their coordinates                                                                       
  supercar_factories = {                                                                                           
      "Ferrari Maranello": (44.5608, 11.3493),                                                                     
      "Lamborghini Sant'Agata Bolognese": (44.7013, 11.3190),                                                      
      "Porsche Leipzig": (51.3797, 12.1910),                                                                       
      "Pagani San Cesario sul Panaro": (44.5823, 10.3399),                                                         
      "Aston Martin Gaydon": (52.2653, -0.7241)                                                                    
  }                                                                                                                
                                                                                                                   
  # Calculate travel time from each supercar factory to Gotham                                                     
  supercar_travel_times = []                                                                                       
  for location, coords in supercar_factories.items():                                                              
      travel_time = calculate_cargo_travel_time(origin_coords=coords, destination_coords=gotham_coords)            
      supercar_travel_times.append(travel_time)                                                                    
                                                                                                                   
  # Create DataFrame for supercar factories                                                                        
  supercar_df = pd.DataFrame({                                                                                     
      'Location': list(supercar_factories.keys()),                                                                 
      'Coordinates': list(supercar_factories.values()),                                                            
      'Travel Time to Gotham (hours)': supercar_travel_times                                                       
  })                                                                                                               
                                                                                                                   
  # Concatenate the Batman filming locations DataFrame and the supercar factories DataFrame                        
  combined_df = pd.concat([batman_df, supercar_df], ignore_index=True)                                             
                                                                                                                   
  print(combined_df)                                                                                               
  final_answer(combined_df)                                                                                        
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                           Location          Coordinates  \
0                  Two Temple Place   (51.5148, -0.0812)   
1               Wellington's Column   (53.4007, -2.9792)   
2                  St George's Hall   (53.4085, -2.9741)   
3                           Chicago  (41.8781, -87.6298)   
4                 Ferrari Maranello   (44.5608, 11.3493)   
5  Lamborghini Sant'Agata Bolognese    (44.7013, 11.319)   
6                   Porsche Leipzig    (51.3797, 12.191)   
7     Pagani San Cesario sul Panaro   (44.5823, 10.3399)   
8               Aston Martin Gaydon   (52.2653, -0.7241)   

   Travel Time to Gotham (hours)  
0                           9.17  
1                           8.81  
2                           8.81  
3                           2.68  
4                          10.77  
5                          10.76  
6                          10.34  
7                          10.67  
8                           9.07  

Out - Final answer:                            Location          Coordinates  \
0                  Two Temple Place   (51.5148, -0.0812)   
1               Wellington's Column   (53.4007, -2.9792)   
2                  St George's Hall   (53.4085, -2.9741)   
3                           Chicago  (41.8781, -87.6298)   
4                 Ferrari Maranello   (44.5608, 11.3493)   
5  Lamborghini Sant'Agata Bolognese    (44.7013, 11.319)   
6                   Porsche Leipzig    (51.3797, 12.191)   
7     Pagani San Cesario sul Panaro   (44.5823, 10.3399)   
8               Aston Martin Gaydon   (52.2653, -0.7241)   

   Travel Time to Gotham (hours)  
0                           9.17  
1                           8.81  
2                           8.81  
3                           2.68  
4                          10.77  
5                          10.76  
6                          10.34  
7                          10.67  
8                           9.07  

[Step 4: Duration 21.68 seconds| Input tokens: 25,451 | Output tokens: 1,635]

In [12]:
result

,Location,Coordinates,Travel Time to Gotham (hours)
0,Two Temple Place,"(51.5148, -0.0812)",9.17
1,Wellington's Column,"(53.4007, -2.9792)",8.81
2,St George's Hall,"(53.4085, -2.9741)",8.81
3,Chicago,"(41.8781, -87.6298)",2.68
4,Ferrari Maranello,"(44.5608, 11.3493)",10.77
5,Lamborghini Sant'Agata Bolognese,"(44.7013, 11.319)",10.76
6,Porsche Leipzig,"(51.3797, 12.191)",10.34
7,Pagani San Cesario sul Panaro,"(44.5823, 10.3399)",10.67
8,Aston Martin Gaydon,"(52.2653, -0.7241)",9.07


In [13]:
agent.planning_interval = 4

detailed_report = agent.run(f"""
You're an expert analyst. You make comprehensive reports after visiting many websites.
Don't hesitate to search for many queries at once in a for loop.
For each data point that you find, visit the source url to confirm numbers.

{task}
""")

print(detailed_report)

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ You're an expert analyst. You make comprehensive reports after visiting many websites.                          │
│ Don't hesitate to search for many queries at once in a for loop.                                                │
│ For each data point that you find, visit the source url to confirm numbers.                                     │
│                                                                                                                 │
│                                                                                                                 │
│      Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here     │
│ (we're in Gotham, 40.7128° N, 74.0060° W), and return them to me as a pandas dataframe.                         │
│ Also give me some supercar factories with the same cargo plane transfer time.                                   │
│                                                                                                                 │
╰─ HfApiModel - https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud ───────────────────────────────╯

────────────────────────────────────────────────── Initial plan ───────────────────────────────────────────────────
Here is the plan of action that I will follow to solve the task:
```
1. Search for a list of Batman filming locations worldwide.
2. For each Batman filming location, visit the source URL to confirm and gather the exact coordinates.
3. Calculate the cargo plane travel time from each Batman filming location to Gotham.
4. Search for a list of supercar factories worldwide.
5. For each supercar factory, visit the source URL to confirm and gather the exact coordinates.
6. Calculate the cargo plane travel time from each supercar factory to Gotham.
7. Filter supercar factories so that they have the same cargo plane transfer time as any of the Batman filming 
locations.
8. Organize all the Batman filming locations and the filtered supercar factories into a pandas dataframe.
9. Final answer: Return the pandas dataframe.

<end_plan>
```

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
  import re                                                                                                        
                                                                                                                   
  # Gotham coordinates                                                                                             
  gotham_coords = (40.7128, -74.0060)                                                                              
                                                                                                                   
  # Search for Batman filming locations                                                                            
  search_queries = [                                                                                               
      "Batman filming locations worldwide",                                                                        
      "where was Batman filmed",                                                                                   
      "Batman movie filming locations"                                                                             
  ]                                                                                                                
                                                                                                                   
  # Store search results                                                                                           
  batman_locations = []                                                                                            
                                                                                                                   
  for query in search_queries:                                                                                     
      results = web_search(query=query)                                                                            
      batman_locations.extend(results.split('\n'))                                                                 
                                                                                                                   
  # Remove duplicates                                                                                              
  batman_locations = list(pd.unique(batman_locations))                                                             
                                                                                                                   
  print(batman_locations)                                                                                          
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

/usr/local/lib/python3.11/dist-packages/smolagents/local_python_executor.py:716: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  return func(*args, **kwargs)


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 9 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  import pandas as pd                                                                                              
                                                                                                                   
  # Prepare the final answer DataFrame                                                                             
  final_df = filtered_supercar_df.copy()                                                                           
                                                                                                                   
  # Final check to ensure the DataFrame is correct                                                                 
  print(final_df)                                                                                                  
                                                                                                                   
  # Final answer                                                                                                   
  final_answer(final_df.to_dict(orient='records'))                                                                 
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
                                            Location          Coordinates  \
0                          Ferrari, Maranello, Italy    (44.596, 11.0736)   
1           Lamborghini, Sant'Agata Bolognese, Italy    (44.549, 11.3665)   
2                      Koenigsegg, Ängelholm, Sweden   (56.4491, 12.8478)   
3                          Porsche, Leipzig, Germany   (51.3067, 12.3354)   
4                                McLaren, Woking, UK   (51.3572, -0.6373)   
5              Pagani, San Cesario sul Panaro, Italy    (45.2123, 10.702)   
6                       Acura, Marysville, Ohio, USA  (40.7432, -83.7562)   
7                             Rimac, Zagreb, Croatia    (45.794, 15.9779)   
8                          Ashton Martin, Gaydon, UK   (52.2219, -1.7237)   
9  Hennessey Performance Engineering, Richmond, T...  (29.6697, -95.5635)   

   Distance to Gotham (km)  Cargo Plane Transfer Time (hours)  
0              6640.354662                           8.019752  
1              6663.168992                           8.047306  
2              6169.905717                           7.451577  
3              6378.323168                           7.703289  
4              5541.900478                           6.693117  
5              6581.726700                           7.948945  
6               821.187259                           0.991772  
7              6906.727776                           8.341459  
8              5441.794489                           6.572216  
9              2302.293434                           2.780548  

Out - Final answer: [{'Location': 'Ferrari, Maranello, Italy', 'Coordinates': (44.596, 11.0736), 'Distance to 
Gotham (km)': 6640.354662490861, 'Cargo Plane Transfer Time (hours)': 8.019752007839205}, {'Location': 
"Lamborghini, Sant'Agata Bolognese, Italy", 'Coordinates': (44.549, 11.3665), 'Distance to Gotham (km)': 
6663.168992172412, 'Cargo Plane Transfer Time (hours)': 8.047305546101947}, {'Location': 'Koenigsegg, Ängelholm, 
Sweden', 'Coordinates': (56.4491, 12.8478), 'Distance to Gotham (km)': 6169.905716999168, 'Cargo Plane Transfer 
Time (hours)': 7.451576952897546}, {'Location': 'Porsche, Leipzig, Germany', 'Coordinates': (51.3067, 12.3354), 
'Distance to Gotham (km)': 6378.323168079406, 'Cargo Plane Transfer Time (hours)': 7.703288850337447}, {'Location':
'McLaren, Woking, UK', 'Coordinates': (51.3572, -0.6373), 'Distance to Gotham (km)': 5541.900478379156, 'Cargo 
Plane Transfer Time (hours)': 6.693116519781589}, {'Location': 'Pagani, San Cesario sul Panaro, Italy', 
'Coordinates': (45.2123, 10.702), 'Distance to Gotham (km)': 6581.726700410159, 'Cargo Plane Transfer Time 
(hours)': 7.948945290350433}, {'Location': 'Acura, Marysville, Ohio, USA', 'Coordinates': (40.7432, -83.7562), 
'Distance to Gotham (km)': 821.1872591756686, 'Cargo Plane Transfer Time (hours)': 0.9917720521445273}, 
{'Location': 'Rimac, Zagreb, Croatia', 'Coordinates': (45.794, 15.9779), 'Distance to Gotham (km)': 
6906.727775758282, 'Cargo Plane Transfer Time (hours)': 8.341458666374738}, {'Location': 'Ashton Martin, Gaydon, 
UK', 'Coordinates': (52.2219, -1.7237), 'Distance to Gotham (km)': 5441.79448870859, 'Cargo Plane Transfer Time 
(hours)': 6.572215566073176}, {'Location': 'Hennessey Performance Engineering, Richmond, TX, USA', 'Coordinates': 
(29.6697, -95.5635), 'Distance to Gotham (km)': 2302.2934341152904, 'Cargo Plane Transfer Time (hours)': 
2.7805476257431043}]

[Step 9: Duration 103.08 seconds| Input tokens: 126,623 | Output tokens: 6,812]

[{'Location': 'Ferrari, Maranello, Italy', 'Coordinates': (44.596, 11.0736), 'Distance to Gotham (km)': 6640.354662490861, 'Cargo Plane Transfer Time (hours)': 8.019752007839205}, {'Location': "Lamborghini, Sant'Agata Bolognese, Italy", 'Coordinates': (44.549, 11.3665), 'Distance to Gotham (km)': 6663.168992172412, 'Cargo Plane Transfer Time (hours)': 8.047305546101947}, {'Location': 'Koenigsegg, Ängelholm, Sweden', 'Coordinates': (56.4491, 12.8478), 'Distance to Gotham (km)': 6169.905716999168, 'Cargo Plane Transfer Time (hours)': 7.451576952897546}, {'Location': 'Porsche, Leipzig, Germany', 'Coordinates': (51.3067, 12.3354), 'Distance to Gotham (km)': 6378.323168079406, 'Cargo Plane Transfer Time (hours)': 7.703288850337447}, {'Location': 'McLaren, Woking, UK', 'Coordinates': (51.3572, -0.6373), 'Distance to Gotham (km)': 5541.900478379156, 'Cargo Plane Transfer Time (hours)': 6.693116519781589}, {'Location': 'Pagani, San Cesario sul Panaro, Italy', 'Coordinates': (45.2123, 10.702), 

In [14]:
detailed_report

[{'Location': 'Ferrari, Maranello, Italy',
  'Coordinates': (44.596, 11.0736),
  'Distance to Gotham (km)': 6640.354662490861,
  'Cargo Plane Transfer Time (hours)': 8.019752007839205},
 {'Location': "Lamborghini, Sant'Agata Bolognese, Italy",
  'Coordinates': (44.549, 11.3665),
  'Distance to Gotham (km)': 6663.168992172412,
  'Cargo Plane Transfer Time (hours)': 8.047305546101947},
 {'Location': 'Koenigsegg, Ängelholm, Sweden',
  'Coordinates': (56.4491, 12.8478),
  'Distance to Gotham (km)': 6169.905716999168,
  'Cargo Plane Transfer Time (hours)': 7.451576952897546},
 {'Location': 'Porsche, Leipzig, Germany',
  'Coordinates': (51.3067, 12.3354),
  'Distance to Gotham (km)': 6378.323168079406,
  'Cargo Plane Transfer Time (hours)': 7.703288850337447},
 {'Location': 'McLaren, Woking, UK',
  'Coordinates': (51.3572, -0.6373),
  'Distance to Gotham (km)': 5541.900478379156,
  'Cargo Plane Transfer Time (hours)': 6.693116519781589},
 {'Location': 'Pagani, San Cesario sul Panaro, Italy',

In [16]:
# improving system structure to avoid it being slower
#splitting the task between two agents
# Each agent will be more focused on its task
# Separating memories reduces count of input tokens

model = HfApiModel(model_id='https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud')

web_agent = CodeAgent(
    model = model,
    tools = [
        DuckDuckGoSearchTool(),
        VisitWebpageTool(),
        calculate_cargo_travel_time,
    ],

    name = "web_agent",
    description = "Browses the web to find information",
    verbosity_level = 0,
    max_steps =10,
)


In [18]:
from smolagents.utils import encode_image_base64, make_image_url
from smolagents import OpenAIServerModel


def check_reasoning_and_plot(final_answer, agent_memory):
    multimodal_model = OpenAIServerModel("gpt-4o", max_tokens=8096)
    filepath = "saved_map.png"
    assert os.path.exists(filepath), "Make sure to save the plot under saved_map.png!"
    image = Image.open(filepath)
    prompt = (
        f"Here is a user-given task and the agent steps: {agent_memory.get_succinct_steps()}. Now here is the plot that was made."
        "Please check that the reasoning process and plot are correct: do they correctly answer the given task?"
        "First list reasons why yes/no, then write your final decision: PASS in caps lock if it is satisfactory, FAIL if it is not."
        "Don't be harsh: if the plot mostly solves the task, it should pass."
        "To pass, a plot should be made using px.scatter_map and not any other method (scatter_map looks nicer)."
    )
    messages = [
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": prompt,
                },
                {
                    "type": "image_url",
                    "image_url": {"url": make_image_url(encode_image_base64(image))},
                },
            ],
        }
    ]
    output = multimodal_model(messages).content
    print("Feedback: ", output)
    if "FAIL" in output:
        raise Exception(output)
    return True


manager_agent = CodeAgent(
    model=HfApiModel("deepseek-ai/DeepSeek-R1", provider="together", max_tokens=8096),
    tools=[calculate_cargo_travel_time],
    managed_agents=[web_agent],
    additional_authorized_imports=[
        "geopandas",
        "plotly",
        "shapely",
        "json",
        "pandas",
        "numpy",
    ],
    planning_interval=5,
    verbosity_level=2,
    final_answer_checks=[check_reasoning_and_plot],
    max_steps=15,
)

In [19]:
manager_agent.visualize()

CodeAgent | deepseek-ai/DeepSeek-R1
├── ✅ Authorized imports: ['geopandas', 'plotly', 'shapely', 'json', 'pandas', 'numpy']
├── 🛠️ Tools:
│   ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
│   ┃ Name                        ┃ Description                           ┃ Arguments                             ┃
│   ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   │ calculate_cargo_travel_time │ Calculate the travel time for a cargo │ origin_coords (`array`): Tuple of     │
│   │                             │ plane between two points on Earth     │ (latitude, longitude) for the         │
│   │                             │ using great-circle distance.          │ starting point                        │
│   │                             │                                       │ destination_coords (`array`): Tuple   │
│   │                             │                                       │ of (latitude, longitude) for the      │
│   │                             │                                       │ destination                           │
│   │                             │                                       │ cruising_speed_kmh (`number`):        │
│   │                             │                                       │ Optional cruising speed in km/h       │
│   │                             │                                       │ (defaults to 750 km/h for typical     │
│   │                             │                                       │ cargo planes)                         │
│   │ final_answer                │ Provides a final answer to the given  │ answer (`any`): The final answer to   │
│   │                             │ problem.                              │ the problem                           │
│   └─────────────────────────────┴───────────────────────────────────────┴───────────────────────────────────────┘
└── 🤖 Managed agents:
    └── web_agent | CodeAgent | https://pflgm2locj2t89co.us-east-1.aws.endpoints.huggingface.cloud
        ├── ✅ Authorized imports: []
        ├── 📝 Description: Browses the web to find information
        └── 🛠️ Tools:
            ┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
            ┃ Name                        ┃ Description                       ┃ Arguments                         ┃
            ┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
            │ web_search                  │ Performs a duckduckgo web search  │ query (`string`): The search      │
            │                             │ based on your query (think a      │ query to perform.                 │
            │                             │ Google search) then returns the   │                                   │
            │                             │ top search results.               │                                   │
            │ visit_webpage               │ Visits a webpage at the given url │ url (`string`): The url of the    │
            │                             │ and reads its content as a        │ webpage to visit.                 │
            │                             │ markdown string. Use this to      │                                   │
            │                             │ browse webpages.                  │                                   │
            │ calculate_cargo_travel_time │ Calculate the travel time for a   │ origin_coords (`array`): Tuple of │
            │                             │ cargo plane between two points on │ (latitude, longitude) for the     │
            │                             │ Earth using great-circle          │ starting point                    │
            │                             │ distance.                         │ destination_coords (`array`):     │
            │          

In [20]:
manager_agent.run("""
Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're in Gotham, 40.7128° N, 74.0060° W).
Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in total.
Represent this as spatial map of the world, with the locations represented as scatter points with a color that depends on the travel time, and save it to saved_map.png!

Here's an example of how to plot and return a map:
import plotly.express as px
df = px.data.carshare()
fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,
     color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)
fig.show()
fig.write_image("saved_image.png")
final_answer(fig)

Never try to process strings using code: when you have a string to read, just print it and you'll see it.
""")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ Find all Batman filming locations in the world, calculate the time to transfer via cargo plane to here (we're   │
│ in Gotham, 40.7128° N, 74.0060° W).                                                                             │
│ Also give me some supercar factories with the same cargo plane transfer time. You need at least 6 points in     │
│ total.                                                                                                          │
│ Represent this as spatial map of the world, with the locations represented as scatter points with a color that  │
│ depends on the travel time, and save it to saved_map.png!                                                       │
│                                                                                                                 │
│ Here's an example of how to plot and return a map:                                                              │
│ import plotly.express as px                                                                                     │
│ df = px.data.carshare()                                                                                         │
│ fig = px.scatter_map(df, lat="centroid_lat", lon="centroid_lon", text="name", color="peak_hour", size=100,      │
│      color_continuous_scale=px.colors.sequential.Magma, size_max=15, zoom=1)                                    │
│ fig.show()                                                                                                      │
│ fig.write_image("saved_image.png")                                                                              │
│ final_answer(fig)                                                                                               │
│                                                                                                                 │
│ Never try to process strings using code: when you have a string to read, just print it and you'll see it.       │
│                                                                                                                 │
╰─ HfApiModel - deepseek-ai/DeepSeek-R1 ──────────────────────────────────────────────────────────────────────────╯

[Step 1: Duration 0.00 seconds]

ValueError: You must provide an api_key to work with Together API or log in with `huggingface-cli login`.